In [1]:
import polars as pl

def lazy_reader(columnslf = pl.scan_parquet(source="/app/data/parquet/transactions.parquet",
                     low_memory=True)

Looking to enrich/fill in the data for locations.

In [14]:
columns_to_read = ["city", "state", "zip_code", "latitude", "longitude"]
    
select_df = pl.read_parquet(source="/app/data/parquet/transactions.parquet",
                            columns=columns_to_read,
                            use_pyarrow=True,
                            memory_map=True,
                            low_memory=False)

In [15]:
select_df.head()

city,state,zip_code,latitude,longitude
str,str,i64,f64,f64
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null


Trying to answer how many of the transactions have a city and a state but no zip code. The missing data of city, state, or zip code will be filled in using the list of zip code data downloaded from https://www.unitedstateszipcodes.org/zip-code-database/. Data contains all of the zip codes for the US.

In [9]:
not_null_city_state_null_zip = select_df.filter((pl.col("city").is_not_null()) & pl.col("state").is_not_null() & pl.col("zip_code").is_null())

In [10]:
not_null_city_state_null_zip.head()

city,state,zip_code
str,str,i64
"""Little Rock""","""AR""",null
"""Little Rock""","""AR""",null
"""Little Rock""","""AR""",null
"""Little Rock""","""AR""",null
"""Benton""","""AR""",null


In [12]:
not_null_city_state_null_zip.shape

(14158198, 3)

In [13]:
not_null_city_state_null_zip.filter(pl.col("zip_code").is_not_null())

city,state,zip_code
str,str,i64


In [17]:
bentonville = select_df.filter(pl.col("zip_code") == 72713)

In [18]:
bentonville.head()

city,state,zip_code,latitude,longitude
str,str,i64,f64,f64
"""Bentonville""","""AR""",72713,36.335249,-94.221973
"""Bentonville""","""AR""",72713,36.335249,-94.221973
"""Bentonville""","""AR""",72713,36.275619,-94.297313
"""Bentonville""","""AR""",72713,36.335249,-94.221973
"""Bentonville""","""AR""",72713,36.335249,-94.221973
